In [1]:
#import pytorch_lightning
import torch

Here is a canonical method of creating a sparse tensor.

In [2]:
def createS():
    # These are the indices.  They need to be integers.
    # 2 rows and n columns
    i = torch.tensor([[0, 1, 1],
                      [2, 0, 2]])
    # n scalars in a vector with which to fill in the sparse matrix
    v = torch.tensor([3.0, 4.0, 5.0])
    # This needs i, v, and an explicit size, since you can't
    # infer the size (some row or column might be all zero)
    S = torch.sparse_coo_tensor(i, v, (2, 3),
                                requires_grad=True)
    # Note, there is no detach here on purpose.
    # It raises an error otherwise:
    # RuntimeError: set_indices_and_values_unsafe is not allowed on a Tensor created from .data or .detach().
    # If your intent is to change the metadata of a Tensor (such as sizes / strides / storage / storage_offset)
    # without autograd tracking the change, remove the .data / .detach() call and wrap the change in a `with torch.no_grad():` block.
    # For example, change:
    #   x.data.set_(y)
    # to:
    # with torch.no_grad():
    #     x.set_(y)
    with torch.no_grad():
        # Not working version
        # S = S.coalesce().clone().detach().requires_grad_(True)
        S = S.coalesce().clone().requires_grad_(True)
    return S

We will eventually want an input, and a target since our equation will be
something like:

$$
S b = y
$$

Note, they can be dense since they are just 1-D vectors.

In [3]:
b = torch.tensor([[3.0], [4.0], [5.0]],
                 requires_grad=False)

y = torch.tensor([[2.0], [1.0]],
                 requires_grad=False)

In [4]:
# This is a sparse dot product you can take a gradient
# of.  See this page for details:
# https://pytorch.org/docs/stable/sparse.html
S = createS()
yHat = torch.sparse.mm(S, b)
yHat

tensor([[15.],
        [37.]], grad_fn=<SparseAddmmBackward0>)

In [5]:
# This is the simplest possible example of a gradient descent with all of the
# pieces pulled apart.
S = createS()
for i in range(10):
    if S.grad is not None:
        S.grad.zero_()

    yHat = torch.sparse.mm(S, b)
    loss = torch.norm(y-yHat)
    print(loss)
    loss.backward()

    with torch.no_grad():
        gamma = 0.1
        S -= gamma*S.grad
        # Note
        #    S = S - gamma*S.grad
        # does not work since S will then be a new copy and not in place!

tensor(38.2753, grad_fn=<LinalgVectorNormBackward0>)
tensor(34.9803, grad_fn=<LinalgVectorNormBackward0>)
tensor(31.6903, grad_fn=<LinalgVectorNormBackward0>)
tensor(28.4059, grad_fn=<LinalgVectorNormBackward0>)
tensor(25.1283, grad_fn=<LinalgVectorNormBackward0>)
tensor(21.8586, grad_fn=<LinalgVectorNormBackward0>)
tensor(18.5986, grad_fn=<LinalgVectorNormBackward0>)
tensor(15.3509, grad_fn=<LinalgVectorNormBackward0>)
tensor(12.1193, grad_fn=<LinalgVectorNormBackward0>)
tensor(8.9105, grad_fn=<LinalgVectorNormBackward0>)


In [6]:
S.grad.values()

tensor([2.4394, 2.6187, 4.3645])

In [7]:
# Now we be a bit fancier and use an optimizer.  This is identical to the above
S = createS()
optimizer = torch.optim.SGD([S], lr=0.1)
for i in range(10):
    optimizer.zero_grad()
    yHat = torch.sparse.mm(S, b)
    loss = torch.norm(y-yHat)
    print(loss)
    loss.backward()

    optimizer.step()


tensor(38.2753, grad_fn=<LinalgVectorNormBackward0>)
tensor(34.9803, grad_fn=<LinalgVectorNormBackward0>)
tensor(31.6903, grad_fn=<LinalgVectorNormBackward0>)
tensor(28.4059, grad_fn=<LinalgVectorNormBackward0>)
tensor(25.1283, grad_fn=<LinalgVectorNormBackward0>)
tensor(21.8586, grad_fn=<LinalgVectorNormBackward0>)
tensor(18.5986, grad_fn=<LinalgVectorNormBackward0>)
tensor(15.3509, grad_fn=<LinalgVectorNormBackward0>)
tensor(12.1193, grad_fn=<LinalgVectorNormBackward0>)
tensor(8.9105, grad_fn=<LinalgVectorNormBackward0>)


In [8]:
# Let's play around with momentum
S = createS()
# The velocity vector for the optimization
V = createS()*0
for i in range(10):
    if S.grad is not None:
        S.grad.zero_()

    yHat = torch.sparse.mm(S, b)
    loss = torch.norm(y-yHat)
    print(loss)
    loss.backward()

    with torch.no_grad():
        gamma = 0.1
        mu = 0.1
        # This momentum implementation is from
        #   https://pytorch.org/docs/stable/generated/torch.optim.SGD.html
        V = mu*V + S.grad
        S -= gamma*V
        # Note
        #    S = S - gamma*S.grad
        # does not work since S will then be a new copy and not in place!

tensor(38.2753, grad_fn=<LinalgVectorNormBackward0>)
tensor(34.9803, grad_fn=<LinalgVectorNormBackward0>)
tensor(31.3612, grad_fn=<LinalgVectorNormBackward0>)
tensor(27.7163, grad_fn=<LinalgVectorNormBackward0>)
tensor(24.0766, grad_fn=<LinalgVectorNormBackward0>)
tensor(20.4472, grad_fn=<LinalgVectorNormBackward0>)
tensor(16.8311, grad_fn=<LinalgVectorNormBackward0>)
tensor(13.2328, grad_fn=<LinalgVectorNormBackward0>)
tensor(9.6599, grad_fn=<LinalgVectorNormBackward0>)
tensor(6.1281, grad_fn=<LinalgVectorNormBackward0>)


Note, this one does not work! This is because pytorch optimizers are not really setup
for running sparse tensors... at least the way I think of them.

- https://github.com/pytorch/pytorch/issues/1285
- https://github.com/pytorch/pytorch/issues/1369
- https://github.com/pytorch/pytorch/issues/9674
- https://github.com/pytorch/pytorch/issues/10043

Here is a search for all of the discussion

https://github.com/pytorch/pytorch/labels/module%3A%20sparse

In [37]:
b = torch.tensor([[3.0], [4.0], [5.0]],
                 requires_grad=False)

y = torch.tensor([[2.0], [1.0]],
                 requires_grad=False)

i = torch.tensor([[0, 1, 1],
                  [2, 0, 2]])
# n scalars in a vector with which to fill in the sparse matrix
v = torch.tensor([3.0, 4.0, 5.0])
# This needs i, v, and an explicit size, since you can't
# infer the size (some row or column might be all zero)
S = torch.sparse_coo_tensor(i, v, (2, 3),
                            requires_grad=True).coalesce()

class SparseModel(torch.nn.Module):
    def __init__(self, S, b):
        super().__init__()
        self.S = torch.nn.Parameter(S)

    def forward(self, b):
        return torch.sparse.mm(self.S, b)
    
my_model = SparseModel(S, b)


optimizer = torch.optim.Adagrad(my_model.parameters(), lr=0.1)
# optimizer = torch.optim.SparseAdam(my_model.parameters(), lr=0.1)
# optimizer = torch.optim.SGD(my_model.parameters(), lr=0.1)

optimizer.zero_grad()
yHat = my_model(b)
loss = torch.norm(y-yHat)
print(loss)
loss.backward()

optimizer.step()


RuntimeError: memory format option is only supported by strided tensors

In [22]:
make_error = True
if make_error:
    S = createS()
    optimizer = torch.optim.Adagrad([S], lr=0.1)
    for i in range(10):
        optimizer.zero_grad()
        yHat = torch.sparse.mm(S, b)
        print(b)
        loss = torch.norm(y-yHat)
        print(loss)
        loss.backward()

        optimizer.step()



RuntimeError: memory format option is only supported by strided tensors

Using my little trick.  Takes more memory, but allows optimizers to work

$U$ is an update that you can train, but it goes through a mask $M$ which you can't train


In [10]:
Updates = torch.tensor([[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]], requires_grad=True)
Omega = torch.tensor([[1, 0, 0], [1 , 1, 0]], requires_grad=False)
Init = torch.tensor([[1, 1, 1], [1, 1, 1]], requires_grad=False)

optimizer = torch.optim.Adam([Updates], lr=0.1)
for i in range(10):
    optimizer.zero_grad()
    W = Init+Omega*Updates
    yHat = W @ b
    loss = torch.norm(y-yHat)
    print(loss)
    loss.backward()

    optimizer.step()
print(Updates)
print(Init+Omega*Updates)

tensor(14.8661, grad_fn=<LinalgVectorNormBackward0>)
tensor(14.1485, grad_fn=<LinalgVectorNormBackward0>)
tensor(13.4359, grad_fn=<LinalgVectorNormBackward0>)
tensor(12.7292, grad_fn=<LinalgVectorNormBackward0>)
tensor(12.0295, grad_fn=<LinalgVectorNormBackward0>)
tensor(11.3382, grad_fn=<LinalgVectorNormBackward0>)
tensor(10.6569, grad_fn=<LinalgVectorNormBackward0>)
tensor(9.9876, grad_fn=<LinalgVectorNormBackward0>)
tensor(9.3328, grad_fn=<LinalgVectorNormBackward0>)
tensor(8.6955, grad_fn=<LinalgVectorNormBackward0>)
tensor([[-1.0066,  0.0000,  0.0000],
        [-0.9901, -0.9901,  0.0000]], requires_grad=True)
tensor([[-0.0066,  1.0000,  1.0000],
        [ 0.0099,  0.0099,  1.0000]], grad_fn=<AddBackward0>)


Copy the Pytorch RMSprop and see if I can get it to work with a sparse tensor
https://pytorch.org/docs/stable/generated/torch.optim.RMSprop.html

Here is the raw code
https://raw.githubusercontent.com/pytorch/pytorch/master/torch/optim/rmsprop.py

Here is my slightly modified code.

In [11]:
import torch
from torch import Tensor
from torch.optim import Optimizer
from typing import List, Optional


class SparseRMSprop(Optimizer):
    r"""Implements RMSprop algorithm.

    .. math::
       \begin{aligned}
            &\rule{110mm}{0.4pt}                                                                 \\
            &\textbf{input}      : \alpha \text{ (alpha)},\: \gamma \text{ (lr)},
                \: \theta_0 \text{ (params)}, \: f(\theta) \text{ (objective)}                   \\
            &\hspace{13mm}   \lambda \text{ (weight decay)},\: \mu \text{ (momentum)},\: centered\\
            &\textbf{initialize} : v_0 \leftarrow 0 \text{ (square average)}, \:
                \textbf{b}_0 \leftarrow 0 \text{ (buffer)}, \: g^{ave}_0 \leftarrow 0     \\[-1.ex]
            &\rule{110mm}{0.4pt}                                                                 \\
            &\textbf{for} \: t=1 \: \textbf{to} \: \ldots \: \textbf{do}                         \\
            &\hspace{5mm}g_t           \leftarrow   \nabla_{\theta} f_t (\theta_{t-1})           \\
            &\hspace{5mm}if \: \lambda \neq 0                                                    \\
            &\hspace{10mm} g_t \leftarrow g_t + \lambda  \theta_{t-1}                            \\
            &\hspace{5mm}v_t           \leftarrow   \alpha v_{t-1} + (1 - \alpha) g^2_t
                \hspace{8mm}                                                                     \\
            &\hspace{5mm} \tilde{v_t} \leftarrow v_t                                             \\
            &\hspace{5mm}if \: centered                                                          \\
            &\hspace{10mm} g^{ave}_t \leftarrow g^{ave}_{t-1} \alpha + (1-\alpha) g_t            \\
            &\hspace{10mm} \tilde{v_t} \leftarrow \tilde{v_t} -  \big(g^{ave}_{t} \big)^2        \\
            &\hspace{5mm}if \: \mu > 0                                                           \\
            &\hspace{10mm} \textbf{b}_t\leftarrow \mu \textbf{b}_{t-1} +
                g_t/ \big(\sqrt{\tilde{v_t}} +  \epsilon \big)                                   \\
            &\hspace{10mm} \theta_t \leftarrow \theta_{t-1} - \gamma \textbf{b}_t                \\
            &\hspace{5mm} else                                                                   \\
            &\hspace{10mm}\theta_t      \leftarrow   \theta_{t-1} -
                \gamma  g_t/ \big(\sqrt{\tilde{v_t}} + \epsilon \big)  \hspace{3mm}              \\
            &\rule{110mm}{0.4pt}                                                          \\[-1.ex]
            &\bf{return} \:  \theta_t                                                     \\[-1.ex]
            &\rule{110mm}{0.4pt}                                                          \\[-1.ex]
       \end{aligned}

    For further details regarding the algorithm we refer to
    `lecture notes <https://www.cs.toronto.edu/~tijmen/csc321/slides/lecture_slides_lec6.pdf>`_ by G. Hinton.
    and centered version `Generating Sequences
    With Recurrent Neural Networks <https://arxiv.org/pdf/1308.0850v5.pdf>`_.
    The implementation here takes the square root of the gradient average before
    adding epsilon (note that TensorFlow interchanges these two operations). The effective
    learning rate is thus :math:`\gamma/(\sqrt{v} + \epsilon)` where :math:`\gamma`
    is the scheduled learning rate and :math:`v` is the weighted moving average
    of the squared gradient.

    Args:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float, optional): learning rate (default: 1e-2)
        momentum (float, optional): momentum factor (default: 0)
        alpha (float, optional): smoothing constant (default: 0.99)
        eps (float, optional): term added to the denominator to improve
            numerical stability (default: 1e-8)
        centered (bool, optional) : if ``True``, compute the centered RMSProp,
            the gradient is normalized by an estimation of its variance
        weight_decay (float, optional): weight decay (L2 penalty) (default: 0)
        foreach (bool, optional): whether foreach implementation of optimizer
            is used (default: None)

    """

    def __init__(self, params, lr=1e-2, alpha=0.99, eps=1e-8, weight_decay=0, momentum=0,
                 centered=False, foreach: Optional[bool] = None):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= momentum:
            raise ValueError("Invalid momentum value: {}".format(momentum))
        if not 0.0 <= weight_decay:
            raise ValueError("Invalid weight_decay value: {}".format(weight_decay))
        if not 0.0 <= alpha:
            raise ValueError("Invalid alpha value: {}".format(alpha))

        defaults = dict(lr=lr, momentum=momentum, alpha=alpha, eps=eps, centered=centered,
                        weight_decay=weight_decay, foreach=foreach)
        super(SparseRMSprop, self).__init__(params, defaults)

    def __setstate__(self, state):
        super().__setstate__(state)
        for group in self.param_groups:
            group.setdefault('momentum', 0)
            group.setdefault('centered', False)
            group.setdefault('foreach', None)

    @torch.no_grad()
    def step(self, closure=None):
        """Performs a single optimization step.

        Args:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        for group in self.param_groups:
            params_with_grad = []
            grads = []
            square_avgs = []
            grad_avgs = []
            momentum_buffer_list = []

            for p in group['params']:
                if p.grad is None:
                    continue
                params_with_grad.append(p)

                grads.append(p.grad)

                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    # FIXME:  This is not quite what I intended, even though it works.  zeros_like of a sparse
                    # matrix is an empty matrix.  It might be okay to just use zeros_like since it does the
                    # right thing for dense matrices, and it just an empty matrix for sparse matrices.
                    state['square_avg'] = torch.zeros_like(p)
                    if group['momentum'] > 0:
                        state['momentum_buffer'] = torch.zeros_like(p)
                    if group['centered']:
                        state['grad_avg'] = torch.zeros_like(p)

                square_avgs.append(state['square_avg'])

                if group['momentum'] > 0:
                    momentum_buffer_list.append(state['momentum_buffer'])
                if group['centered']:
                    grad_avgs.append(state['grad_avg'])

                state['step'] += 1


            rmsprop(params_with_grad,
                    grads,
                    square_avgs,
                    grad_avgs,
                    momentum_buffer_list,
                    lr=group['lr'],
                    alpha=group['alpha'],
                    eps=group['eps'],
                    weight_decay=group['weight_decay'],
                    momentum=group['momentum'],
                    centered=group['centered'],
                    foreach=group['foreach'])

        return loss


def rmsprop(params: List[Tensor],
            grads: List[Tensor],
            square_avgs: List[Tensor],
            grad_avgs: List[Tensor],
            momentum_buffer_list: List[Tensor],
            # kwonly args with defaults are not supported by functions compiled with torchscript issue #70627
            # setting this as kwarg for now as functional API is compiled by torch/distributed/optim
            foreach: bool = None,
            *,
            lr: float,
            alpha: float,
            eps: float,
            weight_decay: float,
            momentum: float,
            centered: bool):
    r"""Functional API that performs rmsprop algorithm computation.
    See :class:`~torch.optim.RMSProp` for details.
    """

    if foreach is None:
        # Placeholder for more complex foreach logic to be added when value is not set
        foreach = False

    assert not foreach, 'Not supported'

    if foreach and torch.jit.is_scripting():
        raise RuntimeError('torch.jit.script not supported with foreach optimizers')

    if foreach and not torch.jit.is_scripting():
        func = _multi_tensor_rmsprop
    else:
        func = _single_tensor_rmsprop

    func(params,
         grads,
         square_avgs,
         grad_avgs,
         momentum_buffer_list,
         lr=lr,
         alpha=alpha,
         eps=eps,
         weight_decay=weight_decay,
         momentum=momentum,
         centered=centered)


def _single_tensor_rmsprop(params: List[Tensor],
                           grads: List[Tensor],
                           square_avgs: List[Tensor],
                           grad_avgs: List[Tensor],
                           momentum_buffer_list: List[Tensor],
                           *,
                           lr: float,
                           alpha: float,
                           eps: float,
                           weight_decay: float,
                           momentum: float,
                           centered: bool):

    for i, param in enumerate(params):
        grad = grads[i]
        square_avg = square_avgs[i]

        if weight_decay != 0:
            grad = grad.add(param, alpha=weight_decay)

        if square_avg.is_sparse:
            square_avg *= alpha
            square_avg += grad*grad*(1-alpha)
        else:
            square_avg.mul_(alpha).addcmul_(grad, grad, value=1 - alpha)

        if centered:
            grad_avg = grad_avgs[i]
            assert not grad_avg.is_sparse, 'Not supported'
            grad_avg.mul_(alpha).add_(grad, alpha=1 - alpha)
            avg = square_avg.addcmul(grad_avg, grad_avg, value=-1).sqrt_().add_(eps)
        else:
            if square_avg.is_sparse:
                avg = square_avg.sqrt()
                # Note: tmp is a pointer to the same memory as the values of avg, so updating
                # tmp propagates back to 
                tmp = avg.values()
                tmp += eps
            else:
                avg = square_avg.sqrt().add_(eps)

        if momentum > 0:
            buf = momentum_buffer_list[i]
            assert not buf.is_sparse, 'Not supported'
            buf.mul_(momentum).addcdiv_(grad, avg)
            param.add_(buf, alpha=-lr)
        else:
            # This needs some explanation. These are copies
            # of the original sparse tensors, but just the data
            # part.  They are also "coalesce" versions as in:
            # https://pytorch.org/docs/stable/sparse.html
            # Of course, this does not work with non-coalesced
            # vectors.
            if param.is_sparse:
                grad_v = grad.coalesce().clone().detach().values()
                avg_v = avg.coalesce().clone().detach().values()
                # These need to be coalesced, otherwise this is not defined.
                grad_v /= avg_v
                param += -lr*grad
            else:
                param.addcdiv_(grad, avg, value=-lr)

def _multi_tensor_rmsprop(params: List[Tensor],
                          grads: List[Tensor],
                          square_avgs: List[Tensor],
                          grad_avgs: List[Tensor],
                          momentum_buffer_list: List[Tensor],
                          *,
                          lr: float,
                          alpha: float,
                          eps: float,
                          weight_decay: float,
                          momentum: float,
                          centered: bool):

    if len(params) == 0:
        return

    if weight_decay != 0:
        torch._foreach_add_(grads, params, alpha=weight_decay)

    torch._foreach_mul_(square_avgs, alpha)
    torch._foreach_addcmul_(square_avgs, grads, grads, value=1 - alpha)

    if centered:
        torch._foreach_mul_(grad_avgs, alpha)
        torch._foreach_add_(grad_avgs, grads, alpha=1 - alpha)
        avg = torch._foreach_addcmul(square_avgs, grad_avgs, grad_avgs, value=-1)
        torch._foreach_sqrt_(avg)
        torch._foreach_add_(avg, eps)
    else:
        avg = torch._foreach_sqrt(square_avgs)
        torch._foreach_add_(avg, eps)

    if momentum > 0:
        torch._foreach_mul_(momentum_buffer_list, momentum)
        torch._foreach_addcdiv_(momentum_buffer_list, grads, avg)
        torch._foreach_add_(params, momentum_buffer_list, alpha=-lr)
    else:
        torch._foreach_addcdiv_(params, grads, avg, value=-lr)

In [12]:
# Now we be a bit fancier and use an optimizer.  This is identical to the above
S = createS()
optimizer = SparseRMSprop([S], lr=0.1, momentum=0.00)
for i in range(10):
    optimizer.zero_grad()
    yHat = torch.sparse.mm(S, b)
    loss = torch.norm(y-yHat)
    print(loss)
    loss.backward()

    optimizer.step()


tensor(38.2753, grad_fn=<LinalgVectorNormBackward0>)
tensor(34.9803, grad_fn=<LinalgVectorNormBackward0>)
tensor(31.6903, grad_fn=<LinalgVectorNormBackward0>)
tensor(28.4059, grad_fn=<LinalgVectorNormBackward0>)
tensor(25.1283, grad_fn=<LinalgVectorNormBackward0>)
tensor(21.8586, grad_fn=<LinalgVectorNormBackward0>)
tensor(18.5986, grad_fn=<LinalgVectorNormBackward0>)
tensor(15.3509, grad_fn=<LinalgVectorNormBackward0>)
tensor(12.1193, grad_fn=<LinalgVectorNormBackward0>)
tensor(8.9105, grad_fn=<LinalgVectorNormBackward0>)


FIXME:  Looking above this is the same answer as the SGD in all digits!  I must have broken the RMSProp optimizer.

In [13]:
# Now we be a bit fancier and use an optimizer.  This is identical to the above
S1 = createS()
# /tmp/ipykernel_9374/1282355247.py:4: UserWarning: To copy construct from a tensor, 
# it is recommended to use sourceTensor.clone().detach() or 
# sourceTensor.clone().detach().requires_grad_(True), 
# rather than torch.tensor(sourceTensor).
# So, not this
# S2 = torch.tensor(S1.coalesce(), requires_grad=True)
# but this
S2 = S1.coalesce().clone().detach().requires_grad_(True)
# I am a little worried about the .clone() and memory usage, but 
# that is for another day.

yHat = torch.sparse.mm(S2, b)
loss = torch.norm(y-yHat)
loss.backward()

with torch.no_grad():
    print(S2 + S2.grad)
    print(S2.values())
    # Ok, this seems to work.  I am kind of surprised, but happy.
    # It must be that values() gives a pointer to the internal data.  
    # This makes sense from the efficiency point of view.
    tmp = S2.values() 
    tmp += 1
    print(S2.values())
    # Ok, this also seems to work, likely for the same reason.
    tmp /= tmp*2    
    print(S2.values())


tensor(indices=tensor([[0, 1, 1],
                       [2, 0, 2]]),
       values=tensor([4.6982, 6.8217, 9.7028]),
       size=(2, 3), nnz=3, layout=torch.sparse_coo)
tensor([3., 4., 5.], requires_grad=True)
tensor([4., 5., 6.], requires_grad=True)
tensor([0.5000, 0.5000, 0.5000], requires_grad=True)
